# Masterclass: Clasificación Supervisada (de 0 a producción)
**Caso práctico:** Predicción de mora (mora_30d_next3m) usando `dataset_mora.csv`  
**Autor:** Josef Renato Rodríguez Mallma  
**Objetivo de la clase:** construir un pipeline completo de clasificación (EDA → baseline → modelos → métricas → umbral → interpretabilidad → checklist de producción)



## Agenda de la masterclass
1. ¿Qué es clasificación? (intuición + definición formal)
2. Dataset de mora: carga, diccionario rápido y validación temporal
3. Preparación de datos: tipos, nulos, outliers, encoding, escalado
4. Split correcto (temporal) y por qué **NO** usar split aleatorio en riesgo
5. Baseline: Logistic Regression (con regularización)
6. Modelos no lineales: RandomForest / Gradient Boosting (sklearn)
7. Métricas: Accuracy vs ROC-AUC vs PR-AUC, matriz de confusión, KS, lift
8. Desbalance de clases: class_weight, thresholding, (y cuándo usar oversampling)
9. Calibración y elección de umbral por costo
10. Interpretabilidad: coeficientes (LR) + Permutation Importance
11. Checklist final (anti-leakage, drift, PSI, monitoreo)



## 1) ¿Qué es un problema de clasificación?
En **clasificación** queremos predecir una **clase discreta**.  
Ejemplos:
- Fraude: {0,1}
- Churn: {No, Sí}
- Riesgo: {A,B,C,D} (multiclase)

### 1.1 Definición formal
Tenemos un conjunto de datos con:
- Variables (features): \(X = [x_1, x_2, ..., x_p]\)
- Etiqueta (target): \(y\)

Buscamos una función \(f(X)\) que aproxime \(P(y \mid X)\) o que asigne la clase más probable.

### 1.2 En banca / riesgo
Lo más común es:
- **Binary classification**: default/no default, mora/no mora
- Evaluar con métricas robustas a desbalance: **ROC-AUC, PR-AUC, KS, Lift**
- Validación temporal para evitar **leakage** (fuga de información)


In [ ]:
# 2) Setup: imports y configuración
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve
)

from sklearn.inspection import permutation_importance


In [ ]:
# 3) Carga del dataset
# Nota: asumimos que el archivo está en el mismo directorio del notebook o en una ruta conocida.
# En este entorno (sandbox) está en /mnt/data/dataset_mora.csv

DATA_PATH = "/mnt/data/dataset_mora.csv"
df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
display(df.head())


In [ ]:
# 4) Revisión rápida de esquema
display(df.dtypes)
print("\nNulos por columna:")
display(df.isna().sum().sort_values(ascending=False))


In [ ]:
\
# 5) Target y desbalance
target = "mora_30d_next3m"

# Aseguramos que el target sea binario (0/1)
# Si ya viene como 0/1, esto no cambia nada. Si viniera como "Sí/No", lo mapeamos.
if df[target].dtype == "O":
    df[target] = df[target].map({"No": 0, "Sí": 1, "Si": 1, "NO": 0, "SI": 1}).astype("float")

# Distribución
y = df[target].astype(int)
counts = y.value_counts().sort_index()
rate = counts.get(1, 0) / len(y)

print("Distribución del target (0=no mora, 1=mora):")
display(counts)
print(f"Tasa de mora: {rate:.4f} ({rate*100:.2f}%)")

plt.figure()
plt.bar(["0", "1"], [counts.get(0,0), counts.get(1,0)])
plt.title("Distribución de clases (target)")
plt.xlabel("Clase")
plt.ylabel("N")
plt.show()


In [ ]:
# 6) Validación temporal (MUY importante en riesgo)
# 'periodo' normalmente es YYYYMM. Verifiquemos y ordenemos.
# La idea: entrenar con meses antiguos y testear con meses recientes.

time_col = "periodo"

# Intentamos convertir periodo a entero YYYYMM si es posible
def to_yyyymm(x):
    try:
        # si viene como '202401' o 202401
        return int(str(x)[:6])
    except:
        return np.nan

df[time_col] = df[time_col].apply(to_yyyymm).astype("Int64")

print("periodo min/max:", df[time_col].min(), df[time_col].max())
display(df[[time_col]].head())


In [ ]:
# 7) Split temporal recomendado
# Estrategia simple:
# - Ordenamos por periodo
# - Tomamos últimos periodos como test (holdout)
# - El resto se divide en train/val (también temporal)

df = df.sort_values(time_col).reset_index(drop=True)

unique_periods = df[time_col].dropna().unique()
unique_periods = np.sort(unique_periods)

# Tomaremos ~15% de periodos como test (al final)
n_test_periods = max(1, int(len(unique_periods) * 0.15))
test_periods = unique_periods[-n_test_periods:]

# Del restante, ~15% como validación (al final del train)
train_periods = unique_periods[:-n_test_periods]
n_val_periods = max(1, int(len(train_periods) * 0.15))
val_periods = train_periods[-n_val_periods:]
train_periods = train_periods[:-n_val_periods]

train_df = df[df[time_col].isin(train_periods)].copy()
val_df   = df[df[time_col].isin(val_periods)].copy()
test_df  = df[df[time_col].isin(test_periods)].copy()

print("Periodos -> train:", train_periods[0], "a", train_periods[-1], "(", len(train_periods), "meses )")
print("Periodos -> val  :", val_periods[0], "a", val_periods[-1], "(", len(val_periods), "meses )")
print("Periodos -> test :", test_periods[0], "a", test_periods[-1], "(", len(test_periods), "meses )")

print("\nTamaños:", len(train_df), len(val_df), len(test_df))

def show_rate(name, d):
    r = d[target].mean()
    print(f"{name}: mora rate = {r:.4f} ({r*100:.2f}%)")

show_rate("train", train_df)
show_rate("val", val_df)
show_rate("test", test_df)


In [ ]:
# 8) Definimos features y tipos (numéricas vs categóricas)
# IMPORTANTE: evitamos usar id_cliente como feature.
# periodo puede usarse como feature (tendencia temporal) PERO cuidado: puede inducir leakage si el target cambia por políticas.
# Para esta masterclass lo excluimos como feature y lo usamos solo para split.

drop_cols = ["id_cliente", time_col, target]
X_train = train_df.drop(columns=drop_cols, errors="ignore")
y_train = train_df[target].astype(int)

X_val = val_df.drop(columns=drop_cols, errors="ignore")
y_val = val_df[target].astype(int)

X_test = test_df.drop(columns=drop_cols, errors="ignore")
y_test = test_df[target].astype(int)

# Detectamos columnas categóricas por dtype
cat_cols = [c for c in X_train.columns if X_train[c].dtype == "O"]
num_cols = [c for c in X_train.columns if c not in cat_cols]

print("Num cols:", num_cols)
print("Cat cols:", cat_cols)



## 9) Pipeline de preprocesamiento (industrial)
Buenas prácticas:
- **Imputación** de nulos: mediana (num), moda (cat)
- **Encoding**: OneHot para categóricas
- **Escalado**: StandardScaler solo para modelos sensibles a escala (Logistic, SVM, etc.)
- Empaquetar todo en un **Pipeline** para evitar data leakage


In [ ]:
# 10) Construimos preprocesador con ColumnTransformer

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())  # útil para LogisticRegression
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

preprocess



## 11) Baseline: Logistic Regression (con regularización)
**Por qué LR?**
- Fuerte baseline en riesgo
- Interpretable (coeficientes)
- Rápido y estable
- Produce probabilidades (scoring)

**Regularización**
- `penalty='l2'` (Ridge): reduce varianza, evita overfitting
- `C`: inverso de la fuerza de regularización (más C = menos regularización)

**Desbalance**
- `class_weight='balanced'` ajusta el costo de errores según frecuencia de clases


In [ ]:
# 12) Entrenamiento Logistic Regression (baseline)
lr = LogisticRegression(
    max_iter=200,
    class_weight="balanced",  # ayuda cuando la mora es baja
    solver="lbfgs"
)

lr_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", lr)
])

lr_model.fit(X_train, y_train)

# Probabilidades (score)
val_proba = lr_model.predict_proba(X_val)[:, 1]
test_proba = lr_model.predict_proba(X_test)[:, 1]

# Métricas robustas a desbalance
val_roc = roc_auc_score(y_val, val_proba)
val_pr  = average_precision_score(y_val, val_proba)

test_roc = roc_auc_score(y_test, test_proba)
test_pr  = average_precision_score(y_test, test_proba)

print(f"[LR] VAL  ROC-AUC: {val_roc:.4f} | PR-AUC: {val_pr:.4f}")
print(f"[LR] TEST ROC-AUC: {test_roc:.4f} | PR-AUC: {test_pr:.4f}")


In [ ]:
# 13) Curvas ROC y Precision-Recall
def plot_roc_pr(y_true, proba, title_prefix=""):
    fpr, tpr, _ = roc_curve(y_true, proba)
    prec, rec, _ = precision_recall_curve(y_true, proba)

    plt.figure()
    plt.plot(fpr, tpr)
    plt.plot([0,1], [0,1], linestyle="--")
    plt.title(f"{title_prefix} ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.show()

    plt.figure()
    plt.plot(rec, prec)
    plt.title(f"{title_prefix} Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.show()

plot_roc_pr(y_val, val_proba, "LR VAL")
plot_roc_pr(y_test, test_proba, "LR TEST")



## 14) Umbral (threshold) y matriz de confusión
Muchos modelos entregan **probabilidades**. Para decidir clase 0/1 se elige un umbral \(t\):
- Si \(p \ge t\) ⇒ predecir 1 (mora)
- Si \(p < t\)  ⇒ predecir 0

En riesgo, **no** siempre usamos \(t=0.5\).  
Podemos optimizar por:
- Recall mínimo (no dejar morosos fuera)
- Precision mínimo (no castigar buenos clientes)
- Costo financiero (FN y FP tienen costos diferentes)


In [ ]:
# 15) Selección de umbral por F1 o por Recall objetivo (ejemplo)
def evaluate_threshold(y_true, proba, t):
    pred = (proba >= t).astype(int)
    cm = confusion_matrix(y_true, pred)
    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp + fp + 1e-9)
    recall    = tp / (tp + fn + 1e-9)
    f1 = 2*precision*recall / (precision+recall+1e-9)
    return precision, recall, f1, cm

ts = np.linspace(0.05, 0.95, 19)
rows = []
for t in ts:
    p,r,f1,_ = evaluate_threshold(y_val, val_proba, t)
    rows.append((t,p,r,f1))

thr_df = pd.DataFrame(rows, columns=["threshold","precision","recall","f1"])
display(thr_df.sort_values("f1", ascending=False).head(10))

best_t = thr_df.loc[thr_df["f1"].idxmax(), "threshold"]
print("Best threshold (por F1 en VAL):", best_t)

p,r,f1,cm = evaluate_threshold(y_val, val_proba, best_t)
print(f"VAL @t={best_t:.2f} -> precision={p:.3f}, recall={r:.3f}, f1={f1:.3f}")
print("Confusion matrix [TN FP; FN TP]:\n", cm)



## 16) Modelos no lineales
En datos reales, relaciones pueden ser no lineales (ej: utilización TC vs mora).
Modelos típicos:
- RandomForest (robusto, pero puede ser pesado)
- Gradient Boosting (muy fuerte en tabular)
- LightGBM/XGBoost/CatBoost (state-of-the-art, si están disponibles)

Aquí usamos **HistGradientBoostingClassifier** (sklearn) que suele rendir muy bien.


In [ ]:
# 17) RandomForest y HistGradientBoosting (sklearn)
# Nota: Para árboles no necesitamos escalar, pero usamos el mismo preprocess (imputer + onehot)
# Podemos crear un preprocess alternativo SIN scaler, pero lo mantenemos para simplicidad.

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=30,
    n_jobs=-1,
    class_weight="balanced_subsample",
    random_state=42
)

rf_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", rf)
])

rf_model.fit(X_train, y_train)
rf_val_proba = rf_model.predict_proba(X_val)[:, 1]
rf_test_proba = rf_model.predict_proba(X_test)[:, 1]

print(f"[RF] VAL  ROC-AUC: {roc_auc_score(y_val, rf_val_proba):.4f} | PR-AUC: {average_precision_score(y_val, rf_val_proba):.4f}")
print(f"[RF] TEST ROC-AUC: {roc_auc_score(y_test, rf_test_proba):.4f} | PR-AUC: {average_precision_score(y_test, rf_test_proba):.4f}")

hgb = HistGradientBoostingClassifier(
    learning_rate=0.08,
    max_depth=6,
    max_iter=300,
    random_state=42
)

hgb_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", hgb)
])

hgb_model.fit(X_train, y_train)
hgb_val_proba = hgb_model.predict_proba(X_val)[:, 1]
hgb_test_proba = hgb_model.predict_proba(X_test)[:, 1]

print(f"[HGB] VAL  ROC-AUC: {roc_auc_score(y_val, hgb_val_proba):.4f} | PR-AUC: {average_precision_score(y_val, hgb_val_proba):.4f}")
print(f"[HGB] TEST ROC-AUC: {roc_auc_score(y_test, hgb_test_proba):.4f} | PR-AUC: {average_precision_score(y_test, hgb_test_proba):.4f}")


In [ ]:
# 18) Comparación consolidada
models = [
    ("LogisticRegression", val_proba, test_proba),
    ("RandomForest", rf_val_proba, rf_test_proba),
    ("HistGradientBoosting", hgb_val_proba, hgb_test_proba),
]

rows = []
for name, vp, tp in models:
    rows.append({
        "modelo": name,
        "VAL_ROC_AUC": roc_auc_score(y_val, vp),
        "VAL_PR_AUC": average_precision_score(y_val, vp),
        "TEST_ROC_AUC": roc_auc_score(y_test, tp),
        "TEST_PR_AUC": average_precision_score(y_test, tp),
    })

comp = pd.DataFrame(rows).sort_values("TEST_ROC_AUC", ascending=False)
display(comp)



## 19) Interpretabilidad (sin SHAP)
Si no queremos depender de librerías externas, podemos usar:
- **Coeficientes** (Logistic Regression) para entender dirección e impacto
- **Permutation Importance** para cualquier modelo (impacto en métrica al permutar una variable)

> Nota: con OneHotEncoder, las features se expanden. Abajo mostramos cómo recuperar nombres.


In [ ]:
# 20) Recuperar nombres de features después del preprocess (OneHot)
# Esto nos permite ver coeficientes o importancias por feature transformada.

def get_feature_names(preprocessor, num_cols, cat_cols):
    # num
    num_features = num_cols
    # cat
    ohe = preprocessor.named_transformers_["cat"].named_steps["onehot"]
    cat_features = ohe.get_feature_names_out(cat_cols).tolist()
    return num_features + cat_features

# Entrenamos el preprocesador del pipeline LR para poder extraer nombres
pre = lr_model.named_steps["preprocess"]
pre.fit(X_train)

feature_names = get_feature_names(pre, num_cols, cat_cols)
print("N features after encoding:", len(feature_names))
feature_names[:20]


In [ ]:
# 21) Coeficientes de Logistic Regression (top positivos y negativos)
# Importante: coeficientes están en escala estandarizada por StandardScaler.

lr_est = lr_model.named_steps["model"]
coefs = lr_est.coef_.ravel()

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs
}).sort_values("coef", ascending=False)

display(coef_df.head(15))
display(coef_df.tail(15))


In [ ]:
# 22) Permutation Importance (usando ROC-AUC en VALIDACIÓN)
# OJO: permutation_importance requiere un estimator ya entrenado y datos transformables.
# Usaremos el pipeline completo. Scoring='roc_auc'.

result = permutation_importance(
    hgb_model, X_val, y_val,
    n_repeats=5,
    random_state=42,
    scoring="roc_auc"
)

imp_df = pd.DataFrame({
    "feature": X_val.columns,
    "importance_mean": result.importances_mean,
    "importance_std": result.importances_std
}).sort_values("importance_mean", ascending=False)

display(imp_df.head(15))

plt.figure()
plt.barh(imp_df.head(15)["feature"][::-1], imp_df.head(15)["importance_mean"][::-1])
plt.title("Top 15 Permutation Importance (HGB, ROC-AUC en VAL)")
plt.xlabel("Decrease in ROC-AUC (mean)")
plt.ylabel("Feature")
plt.show()



## 23) Checklist de producción (lo que haría un equipo de riesgo / MLOps)

### Anti-leakage
- Verificar que ninguna variable usa información del futuro (post-evento)
- Validación temporal obligatoria

### Estabilidad temporal / drift
- PSI por variable (train vs. val/test vs. producción)
- Monitoreo de tasa de mora por cohorte y segmento

### Calibración y umbrales
- Elegir umbral por costo (FN suele costar más que FP)
- Calibrar probabilidades (Platt/Isotonic) si el score se usa como probabilidad real

### Trazabilidad
- Guardar: dataset version, features, hiperparámetros, métricas, fecha, código
- Registrar decisioning: política de corte (threshold) y excepciones

### Gobernanza
- Explicabilidad mínima: top drivers por segmento
- Documentación: diccionario + supuestos + limitaciones
